In [ ]:
!pip install numpy==1.26.4 scipy==1.13.1 gpytoolbox==0.2.0 polyscope==2.2.1 libigl

## Introduction to Mesh Parameterization: Follow-Along Coding 
In this notebook we will run through the computations that you will be asked to perform in your exercises, on a set of simple meshes.


We will first demonstrate how to perform a trivial (triangle soup) parameterization on the simplest mesh possible -- a single triangle, and verify that the trivial parameterization is in fact distortionless.


By the time you run through this notebook, you should have a better understanding of how to set up and solve the linear system for computing the Tutte embedding, as well as how to alter the system for different choices of Laplacian weights. 


Furthermore, you should know how to compute the Jacobian of the parameterization, obtain the singular values, measure the different types of distortion, and visualize them. 

### Part 1: Single Triangle Parameterization ###
In the case we want to obtain a perfect parameterization of every triangle in the mesh, and we don't care about cuts, then we can always use the trivial parameterization, which projects each vertex of a triangle onto its local basis (see figure below). 

![image](local_basis.png)

In the above image $x_i$ is set to the origin of the local basis, and the triangle edge $x_j - x_i$ is chosen as the first basis vector $X$. From there, the normal is computed as the cross product between the two triangle edges connected to $x_i$, and another cross product is taken to get the second basis vector $Y$. 

After these local (2D) bases are defined, we can find the coordinates of each triangle vertex in terms of these bases using the dot product. Specifically, we take the dot product between the edge vectors of each vertex with $X$, $Y$ respectively to get the local coordinates $(u, v)$, which are the 2D parameterization coordinates. The parameterization coordinates for the triangle in the figure are explicitly written out below. 

$$\begin{align*}
u_i = (0, 0) \text{ since } v_i \text{ is the origin in the local basis} \\
u_j = ((x_j - x_i) \cdot X, (x_j - x_i) \cdot Y) \\
u_k = ((x_k - x_i) \cdot X, (x_k - x_i) \cdot Y) \\
\end{align*}$$

In [ ]:
import numpy as np

# This function computes a trivial parameterization of a single triangle using the technique described above.
def trivial_parameterization(triangle):
    # Local basis origin is the first triangle vertex
    origin = triangle[0]

    # Get the normal of the plane defined by the triangle
    e1 = triangle[1] - origin
    e2 = triangle[2] - origin
    normal = np.cross(e1, e2)
    normal /= np.linalg.norm(normal)

    # We will use e1 as the first basis. Second basis will be cross between e1 and the normal
    basis1 = e1 / np.linalg.norm(e1)
    basis2 = np.cross(normal, basis1)
    basis2 /= np.linalg.norm(basis2)

    # First vertex is the origin. Project the other two vertices onto the local bases.
    uv0 = np.array([0,0])
    uv1 = np.array([np.dot(e1, basis1), np.dot(e1, basis2)])
    uv2 = np.array([np.dot(e2, basis1), np.dot(e2, basis2)])

    return np.array([uv0, uv1, uv2]), origin, basis1, basis2

In [ ]:
# Define a triangle and compute its trivial parameterization
triangle = np.array([[5.1, 2.8, 9.9],
                     [-1, 8, 2.3],
                     [8, 4.4, 6.7]])

uvs, origin, basis1, basis2 = trivial_parameterization(triangle)

The below polyscope code visualizes the original 3D triangle (blue) with the local bases $X$ (red) and $Y$ (blue) plotted. The parameterized triangle (green) is also plotted. 

In [ ]:
# Visualize the bases and the UV coordinates
import polyscope as ps

ps.init()
ps.remove_all_structures()
ps.register_surface_mesh("single tri", triangle, np.array([[0,1,2]]), edge_width=1, color = (0.1, 0.2, 1))
ps.register_curve_network("basis1", np.array([origin, origin + basis1]), np.array([[0,1]]), radius=0.01, color=(1, 0, 0))
ps.register_curve_network("basis2", np.array([origin, origin + basis2]), np.array([[0,1]]), radius=0.01, color=(0, 0, 1))
ps.register_surface_mesh("uv tri", uvs, np.array([[0,1,2]]), edge_width=1, color = (0.2, 1, 0.2))
ps.reset_camera_to_home_view()
ps.show()

Let's confirm that the trivial parameterization is indeed distortionless by finding the Jacobian of the mapping. 

This is as simple as getting the gradient operator of this triangle, reshaping a bit, and multiplying it against the UV values. 

In [ ]:
def get_jacobian(vs, fs, uvmap):
    """ Get jacobian of mesh given an input UV map

    Args:
        vs (np.ndarray): V x 3 array of vertex positions
        fs (np.ndarray): F x 3 integer array of face indices
        uvmap (np.ndarray): V x 2 array of UV coordinates

    Returns:
        _type_: _description_
    """
    # Visualize distortion
    from igl import grad
    G = np.array(grad(vs, fs).todense())

    # NOTE: currently gradient is organized as X1, X2, X3, ... Y1, Y2, Y3, ... Z1, Z2, Z3 ... reshape to X1, Y1, Z1, ...
    splitind = G.shape[0]//3
    newG = np.zeros_like(G) # F*3 x V
    newG[::3] = G[:splitind]
    newG[1::3] = G[splitind:2*splitind]
    newG[2::3] = G[2*splitind:]

    J = (newG @ uvmap).reshape(-1, 3, 2) # F x 3 x 2
    return J

J = get_jacobian(triangle, np.array([[0,1,2]]), uvs)

<p align="center"><img src="ss.png" width="600" /></p>

We can use numpy to obtain the singular values of this map. Recall that each face of the mesh gets two singular values, with the distortion measures defined above. 

In [ ]:
_, S, _ = np.linalg.svd(J)

S = S[0] # Get singular values of first (only) face

## Let's confirm there is no distortion
# It is obvious if we just print the values
print("##### Singular values of the trivial parameterization #####")
print(S)

# Area preservation: s1 * s2 == 1
assert np.isclose(S[0] * S[1], 1)

# Conformal (Angle) preservation: s1 == s2
assert np.isclose(S[0], S[1])

# Isometric distortion: s1 == s2 == 1
assert np.isclose(S[0], 1)
assert np.isclose(S[1], 1)

### Part 2: Fixed Boundary Parameterization of Simple Pyramid ###
Now let's take a look at a slightly more difficult example -- an open pyramid (boundary at the base). We will compute fixed boundary parameterizations over this mesh using different choices of Laplacian weights. 

The code below will show how to setup the system of equations to be solved in matrix form, which will allow us to make use of standard Python libraries to solve them. It will also demonstrate how to recompute the parameterization using the same matrices with different sets of weights. Please refer to the exercise writeup to see how the matrix form of this linear system is derived. 

We will investigate how different choices of weights affects both the resulting distortion and the injectivity of the parameterization. 

In [ ]:
import numpy as np
import polyscope as ps

### Define the pyramid and visualize
pyramid_vs = np.array([[3, 0, 1],
                    [-1, -1, 0],
                    [1, -1, 0],
                    [1, 1, 0],
                    [-1, 1, 0]], dtype=np.float32)

pyramid_fs = np.array([[0, 3, 2],
                       [0, 4, 3],
                       [0, 2, 1],
                       [0, 1, 4]], dtype=int)

## Rotate the mesh around to inspect it!
ps.init()
ps.remove_all_structures()
ps.register_surface_mesh("pyramid", pyramid_vs, pyramid_fs, edge_width=1)

# The boundary points are indicated with red spheres
boundaryvs = np.array([pyramid_vs[1], pyramid_vs[2], pyramid_vs[3], pyramid_vs[4]])
boundaryes = np.array([[0,0], [1,1], [2,2], [3,3]])
ps.register_curve_network("boundaries", boundaryvs, boundaryes, radius=0.01, color=(1, 0, 0))

ps.reset_camera_to_home_view()
ps.show()

<p align="center"><img src="pyramid.png" width="400" /></p>

The figure of the pyramid indicates there is an obvious choice of boundary positions we can set. For each of the 4 boundary vertices, we can simply drop the z-coordinate. Once we set the boundary positions, we can proceed to solving the Tutte parameterization problem. Recall that with the Tutte parameterization, all Laplacian (edge) weights are set to 1 ($w_{ij}$ in the below formula). 

$$\min_{\mathbf{U}} \sum_{\{i,j\} \in \mathbf{E}} w_{ij} || \mathbf{u}_i - \mathbf{u}_j||^2$$

In [ ]:
### Find the boundary vertices and fixed positions
# Get the boundary loop using libigl
from igl import boundary_loop

boundary_idxs = list(sorted(boundary_loop(pyramid_fs))) # Get vertex indices of the boundary loop in sorted order
boundary_positions = pyramid_vs[boundary_idxs, :2] # Drop z-axis to get the 2D pinned positions
pred_idxs = np.array([i for i in range(pyramid_vs.shape[0]) if i not in boundary_idxs]) # Get the indices of the interior vertices

#### Build edge array
from collections import defaultdict
edges = defaultdict(int)
for f in pyramid_fs:
    for i in range(3):
        if f[i] > f[(i+1)%3]:
            edges[(f[(i+1)%3], f[i])] += 1
        else:
            edges[(f[i], f[(i+1)%3])] += 1

### Valid (non-boundary) edges are the ones that are shared by two faces
# The valid edges will be how we assign weights to the Laplacian matrix
tot_edges = np.array(list(edges.keys()))
valid_edges = np.array([k for k, v in edges.items() if v == 2])
boundary_edges = np.array([k for k, v in edges.items() if v == 1])

### The below code generates the matrices for the Tutte linear system
def setup_parameterization_matrices(vertices, boundary_idxs, edges, weights=1):
    """ Set up the Tutte linear system (laplacian weights of 1)

    vertices (np.ndarray): V x 3 array of vertex positions
    boundary_idxs (np.ndarray): B array of boundary vertex indices
    edges (np.ndarray): E x 2 array of edge indices for weight assignment
    weights (float or np.ndarray): E array of edge weights or scalar value (default 1)

    Returns:
        L (np.ndarray): V x V Laplacian matrix with boundary values set to 0
        Lb (np.ndarray): V x B boundary Laplacian
    """

    # Initialize the laplacian matrix
    L = np.zeros((vertices.shape[0], vertices.shape[0]))

    L[edges[:, 0], edges[:, 1]] = weights
    L[edges[:, 1], edges[:, 0]] = weights

    # Off-diagonal elements are negative and diagonal is the negative sum of the row
    L = np.diag(np.sum(L, axis=1)) - L

    # Boundary weights are positive
    Lb = -L[:, boundary_idxs]

    # Boundary diagonal should be set to 0
    Lb[boundary_idxs, range(len(boundary_idxs))] = 0

    # Set the off-diagonal boundary columns to 0
    Ldiag = np.diag(L).copy()

    L[:, boundary_idxs] = 0
    np.fill_diagonal(L, Ldiag)

    return L, Lb

L, Lb = setup_parameterization_matrices(pyramid_vs, boundary_idxs, valid_edges)

### Solve the Tutte linear system
# U = L^-1 @ Lb @ Ub
tutte_solution = np.linalg.solve(L, Lb @ boundary_positions)
tutte_uv = np.zeros((pyramid_vs.shape[0], 2))
tutte_uv[pred_idxs] = tutte_solution[pred_idxs]
tutte_uv[boundary_idxs] = boundary_positions

We can now visualize the final parameterization using Polyscope. What do you notice about the UV position of the single non-pinned vertex? Is the parameterization injective (do any triangles overlap)? Is this result expected based on what you learned about the choice of weights? 

In [ ]:
# Visualize the embedding
ps.init()
ps.remove_all_structures()
ps_pyramid = ps.register_surface_mesh("pyramid", pyramid_vs, pyramid_fs, edge_width=1, enabled=True)

# Color the faces to get the correspondence
fcolors = np.arange(pyramid_fs.shape[0])
ps_pyramid.add_scalar_quantity("faces", fcolors, defined_on='faces', cmap='viridis', enabled=True)

# The boundary points are indicated with red spheres
boundaryvs = np.array([pyramid_vs[1], pyramid_vs[2], pyramid_vs[3], pyramid_vs[4]])
boundaryes = np.array([[0,0], [1,1], [2,2], [3,3]])
ps.register_curve_network("boundaries", boundaryvs, boundaryes, radius=0.01, color=(1, 0, 0), enabled=True)

ps_tutte = ps.register_surface_mesh("tutte parameterization", tutte_uv + np.array([[0, 3]]), pyramid_fs, edge_width=1, enabled=True)
ps_tutte.add_scalar_quantity("faces", fcolors, defined_on='faces', cmap='viridis', enabled=True)

ps.reset_camera_to_home_view()
ps.show()

Let's compute and visualize the distortion of the Tutte embedding.

We will define the following energies for area, conformal, and isometric energies: 

$$E_{\text{area}} = \sigma_1\sigma_2 + \frac{1}{\sigma_1\sigma_2} - 2$$

$$E_{\text{conformal}} = \frac{\sigma_1}{\sigma_2} + \frac{\sigma_2}{\sigma_1} - 2$$

$$E_{\text{isometric}} = \sigma_1^2 + \sigma_2^2 + \frac{1}{\sigma_1^2} + \frac{1}{\sigma_2^2} - 4$$

Can you see how they relate to the original definitions for each type of distortion?

In [ ]:
def get_jacobian(vs, fs, uvmap):
    """ Get jacobian of mesh given an input UV map

    Args:
        vs (np.ndarray): V x 3 array of vertex positions
        fs (np.ndarray): F x 3 integer array of face indices
        uvmap (np.ndarray): V x 2 array of UV coordinates

    Returns:
        _type_: _description_
    """
    # Visualize distortion
    from igl import grad
    G = np.array(grad(vs, fs).todense())

    # NOTE: currently gradient is organized as X1, X2, X3, ... Y1, Y2, Y3, ... Z1, Z2, Z3 ... reshape to X1, Y1, Z1, ...
    splitind = G.shape[0]//3
    newG = np.zeros_like(G) # F*3 x V
    newG[::3] = G[:splitind]
    newG[1::3] = G[splitind:2*splitind]
    newG[2::3] = G[2*splitind:]

    J = (newG @ uvmap).reshape(-1, 3, 2) # F x 3 x 2
    return J

J = get_jacobian(pyramid_vs, pyramid_fs, tutte_uv)
_, S, _ = np.linalg.svd(J)

## Area energy
E_area = S[:, 0] * S[:, 1] + 1/(S[:, 0] * S[:, 1]) - 2
E_conformal = (S[:, 0] - S[:, 1])**2
E_isometric = S[:, 0]**2 + S[:, 1]**2 + 1/(S[:, 0]**2) + 1/(S[:, 1]**2) - 4

print("### Tutte distortion energies ###")
print("Mean area energy: ", np.mean(E_area))
print("Mean conformal energy: ", np.mean(E_conformal))
print("Mean isometric energy: ", np.mean(E_isometric))

## Visualize the different distortion values as scalar quantities
ps.init()
ps.remove_all_structures()

ps_pyramid = ps.register_surface_mesh("pyramid", pyramid_vs, pyramid_fs, edge_width=1, enabled=True)

ps_pyramid.add_scalar_quantity("area distortion", E_area, defined_on='faces', cmap='reds', vminmax=(0,2), enabled=True)
ps_pyramid.add_scalar_quantity("conformal distortion", E_conformal, defined_on='faces', cmap='reds', vminmax=(0,2), enabled=False)
ps_pyramid.add_scalar_quantity("isometric distortion", E_isometric, defined_on='faces', cmap='reds', vminmax=(0,10), enabled=False)

ps_tutte = ps.register_surface_mesh("tutte parameterization", tutte_uv + np.array([[0, 3]]), pyramid_fs, edge_width=1, enabled=True)

# vminimax determines the minimum and maximum values for the scalar colormap
ps_tutte.add_scalar_quantity("area distortion", E_area, defined_on='faces', cmap='reds', vminmax=(0,2), enabled=True)
ps_tutte.add_scalar_quantity("conformal distortion", E_conformal, defined_on='faces', cmap='reds', vminmax=(0,2), enabled=False)
ps_tutte.add_scalar_quantity("isometric distortion", E_isometric, defined_on='faces', cmap='reds', vminmax=(0,10), enabled=False)

ps.reset_camera_to_home_view()
ps.show()

Recall the important connection we learned between choice of weights and the injectivity of the parameterization! 

**[Floater 1997] If the pinned bounary is convex and the weights are positive, then the solution to the linear system will be injective.**

Now let's see the effect of a few other choices of weights. 

### Discrete harmonic weights

$$w_{ij} = \cot \gamma_{ij} + \cot \gamma_{ji}$$
$$w_{ij} \text{ is assigned to the edge } (x_i, x_j) \text{ in the diagram below.}$$
<p align="center"><img src="weights.png" width="400" /></p>

In [ ]:
### Compute the discrete harmonic coordinates
# Build a dictionary of cotangents for each half-edge
cotans = {}
for fi in range(len(pyramid_fs)):
    f = pyramid_fs[fi]
    for i in range(3):
        v1 = f[i]
        v2 = f[(i+1)%3]
        v3 = f[(i+2)%3]
        e1 = pyramid_vs[v1] - pyramid_vs[v3]
        e2 = pyramid_vs[v2] - pyramid_vs[v3]

        assert (v1, v2) not in cotans

        # Compute cot(xi, xj)
        cotans[(v1, v2)] = np.dot(e1, e2) / np.linalg.norm(np.cross(e1, e2))

# For each edge pair, compute the harmonic weights
harmonic_weights = np.zeros((len(valid_edges),))
for i, edge in enumerate(valid_edges):
    v1, v2 = edge

    assert (v1, v2) in cotans and (v2, v1) in cotans

    # cot(xi, xj) + cot(xj, xi)
    harmonic_weights[i] = cotans[(v1, v2)] + cotans[(v2, v1)]

L, Lb = setup_parameterization_matrices(pyramid_vs, boundary_idxs, valid_edges, weights=harmonic_weights)

### Solve the Tutte linear system
harmonic_solution = np.linalg.solve(L, Lb @ boundary_positions)

# Bring solution back to final UVs
harmonic_uv = np.zeros_like(pyramid_vs[:, :2])
harmonic_uv[boundary_idxs] = boundary_positions
harmonic_uv[pred_idxs] = harmonic_solution[pred_idxs]

We will now visualize and compare the solutions between the parameterizations with the Tutte weights and the Harmonic weights. The Tutte flattened solution is plotted directly above the pyramid mesh, with the harmonic solution the right of it. 

Is the harmonic solution injective (are any triangles overlapping/flipped)? If so, why? Confirm your answer by looking at the computed harmonic weights. 

In [ ]:
# Compare tutte and harmonic solutions
ps.init()
ps.remove_all_structures()
ps_pyramid = ps.register_surface_mesh("pyramid", pyramid_vs, pyramid_fs, edge_width=1, enabled=True)

# Color the faces to get the correspondence
fcolors = np.arange(pyramid_fs.shape[0])
ps_pyramid.add_scalar_quantity("faces", fcolors, defined_on='faces', cmap='viridis', enabled=True)

# The boundary points are indicated with red spheres
boundaryvs = np.array([pyramid_vs[1], pyramid_vs[2], pyramid_vs[3], pyramid_vs[4]])
boundaryes = np.array([[0,0], [1,1], [2,2], [3,3]])
ps.register_curve_network("boundaries", boundaryvs, boundaryes, radius=0.01, color=(1, 0, 0), enabled=True)

ps_tutte = ps.register_surface_mesh("tutte parameterization", tutte_uv + np.array([[0, 3]]), pyramid_fs, edge_width=1, enabled=True)
ps_tutte.add_scalar_quantity("faces", fcolors, defined_on='faces', cmap='viridis', enabled=True)

ps_harmonic = ps.register_surface_mesh("harmonic parameterization", harmonic_uv + np.array([[3, 3]]), pyramid_fs, edge_width=1, enabled=True)
ps_harmonic.add_scalar_quantity("faces", fcolors, defined_on='faces', cmap='viridis', enabled=True)

ps.reset_camera_to_home_view()
ps.show()

Let's look at the distortion.

In [ ]:
J = get_jacobian(pyramid_vs, pyramid_fs, harmonic_uv)
_, S, _ = np.linalg.svd(J)

## Area energy
E_area = S[:, 0] * S[:, 1] + 1/(S[:, 0] * S[:, 1]) - 2
E_conformal = (S[:, 0] - S[:, 1])**2
E_isometric = S[:, 0]**2 + S[:, 1]**2 + 1/(S[:, 0]**2) + 1/(S[:, 1]**2) - 4

print("### Harmonic distortion energies ###")
print("Mean area energy: ", np.mean(E_area))
print("Mean conformal energy: ", np.mean(E_conformal))
print("Mean isometric energy: ", np.mean(E_isometric))

## Visualize the different distortion values as scalar quantities
ps.init()
ps.remove_all_structures()

ps_pyramid = ps.register_surface_mesh("pyramid", pyramid_vs, pyramid_fs, edge_width=1, enabled=True)

ps_pyramid.add_scalar_quantity("area distortion", E_area, defined_on='faces', cmap='reds', vminmax=(0,2), enabled=True)
ps_pyramid.add_scalar_quantity("conformal distortion", E_conformal, defined_on='faces', cmap='reds', vminmax=(0,2), enabled=False)
ps_pyramid.add_scalar_quantity("isometric distortion", E_isometric, defined_on='faces', cmap='reds', vminmax=(0,10), enabled=False)

ps_tutte = ps.register_surface_mesh("harmonic parameterization", harmonic_uv + np.array([[0, 3]]), pyramid_fs, edge_width=1, enabled=True)

# vminimax determines the minimum and maximum values for the scalar colormap
ps_tutte.add_scalar_quantity("area distortion", E_area, defined_on='faces', cmap='reds', vminmax=(0,2), enabled=True)
ps_tutte.add_scalar_quantity("conformal distortion", E_conformal, defined_on='faces', cmap='reds', vminmax=(0,2), enabled=False)
ps_tutte.add_scalar_quantity("isometric distortion", E_isometric, defined_on='faces', cmap='reds', vminmax=(0,10), enabled=False)

ps.reset_camera_to_home_view()
ps.show()

### Mean value coordinates

$$w_{ij} = \frac{\tan \frac{\alpha_{ij}}{2} + \tan \frac{\beta_{ji}}{2}}{r_{ij}}$$

<p align="center"><img src="weights.png" width="400" /></p>

In [ ]:
### Compute the mean value coordinates
# Build a dictionary of tangents for each half-edge
meanvalues = {}
for fi in range(len(pyramid_fs)):
    f = pyramid_fs[fi]
    for i in range(3):
        v1 = f[i]
        v2 = f[(i+1)%3]
        v3 = f[(i+2)%3]
        e1 = pyramid_vs[v2] - pyramid_vs[v1]
        e2 = pyramid_vs[v3] - pyramid_vs[v1]

        b1 = pyramid_vs[v1] - pyramid_vs[v2]
        b2 = pyramid_vs[v3] - pyramid_vs[v2]

        assert (v1, v2) not in meanvalues

        alpha = np.arccos(np.dot(e1, e2) / (np.linalg.norm(e1) * np.linalg.norm(e2)))
        beta = np.arccos(np.dot(b1, b2) / (np.linalg.norm(b1) * np.linalg.norm(b2)))

        # Compute (tan(alpha_ij/2), tan(beta_ij/2), r_ij)
        meanvalues[(v1, v2)] = (np.tan(alpha/2),
                            np.tan(beta/2),
                            np.linalg.norm(pyramid_vs[v2] - pyramid_vs[v1]))

# For each edge pair, compute the mean value weights
mean_value_weights = np.zeros((len(valid_edges),))
for i, edge in enumerate(valid_edges):
    v1, v2 = edge

    assert (v1, v2) in meanvalues and (v2, v1) in meanvalues

    a1, b2, r1 = meanvalues[(v1, v2)]
    a2, b1, r2 = meanvalues[(v2, v1)]

    assert r1 == r2

    # tan(alpha_ij/2) + tan(beta_ji/2)
    mean_value_weights[i] = (a1 + b1)/r1

L, Lb = setup_parameterization_matrices(pyramid_vs, boundary_idxs, valid_edges, weights=mean_value_weights)

### Solve the Tutte linear system
mean_value_solution = np.linalg.solve(L, Lb @ boundary_positions)

# Bring solution back to final UVs
mean_value_uv = np.zeros_like(pyramid_vs[:, :2])
mean_value_uv[boundary_idxs] = boundary_positions
mean_value_uv[pred_idxs] = mean_value_solution[pred_idxs]

Visually compare all three parameterizations

In [ ]:
# Compare tutte, harmonic, and mean value solutions
ps.init()
ps.remove_all_structures()
ps_pyramid = ps.register_surface_mesh("pyramid", pyramid_vs, pyramid_fs, edge_width=1, enabled=True)

# Color the faces to get the correspondence
fcolors = np.arange(pyramid_fs.shape[0])
ps_pyramid.add_scalar_quantity("faces", fcolors, defined_on='faces', cmap='viridis', enabled=True)

# The boundary points are indicated with red spheres
boundaryvs = np.array([pyramid_vs[1], pyramid_vs[2], pyramid_vs[3], pyramid_vs[4]])
boundaryes = np.array([[0,0], [1,1], [2,2], [3,3]])
ps.register_curve_network("boundaries", boundaryvs, boundaryes, radius=0.01, color=(1, 0, 0), enabled=True)

ps_tutte = ps.register_surface_mesh("tutte parameterization", tutte_uv + np.array([[0, 3]]), pyramid_fs, edge_width=1, enabled=True)
ps_tutte.add_scalar_quantity("faces", fcolors, defined_on='faces', cmap='viridis', enabled=True)

ps_harmonic = ps.register_surface_mesh("harmonic parameterization", harmonic_uv + np.array([[3, 3]]), pyramid_fs, edge_width=1, enabled=True)
ps_harmonic.add_scalar_quantity("faces", fcolors, defined_on='faces', cmap='viridis', enabled=True)

ps_meanvalue = ps.register_surface_mesh("mean value parameterization", mean_value_uv + np.array([[6, 3]]), pyramid_fs, edge_width=1, enabled=True)
ps_meanvalue.add_scalar_quantity("faces", fcolors, defined_on='faces', cmap='viridis', enabled=True)

ps.reset_camera_to_home_view()
ps.show()

Which parameterization is the "best" in terms of the different notions of distortion?

In [ ]:
J = get_jacobian(pyramid_vs, pyramid_fs, mean_value_uv)
_, S, _ = np.linalg.svd(J)

## Area energy
E_area = S[:, 0] * S[:, 1] + 1/(S[:, 0] * S[:, 1]) - 2
E_conformal = (S[:, 0] - S[:, 1])**2
E_isometric = S[:, 0]**2 + S[:, 1]**2 + 1/(S[:, 0]**2) + 1/(S[:, 1]**2) - 4

print("### Mean value distortion energies ###")
print("Mean area energy: ", np.mean(E_area))
print("Mean conformal energy: ", np.mean(E_conformal))
print("Mean isometric energy: ", np.mean(E_isometric))

## Visualize the different distortion values as scalar quantities
ps.init()
ps.remove_all_structures()

ps_pyramid = ps.register_surface_mesh("pyramid", pyramid_vs, pyramid_fs, edge_width=1, enabled=True)

ps_pyramid.add_scalar_quantity("area distortion", E_area, defined_on='faces', cmap='reds', vminmax=(0,2), enabled=True)
ps_pyramid.add_scalar_quantity("conformal distortion", E_conformal, defined_on='faces', cmap='reds', vminmax=(0,2), enabled=False)
ps_pyramid.add_scalar_quantity("isometric distortion", E_isometric, defined_on='faces', cmap='reds', vminmax=(0,10), enabled=False)

ps_tutte = ps.register_surface_mesh("harmonic parameterization", mean_value_uv + np.array([[0, 3]]), pyramid_fs, edge_width=1, enabled=True)

# vminimax determines the minimum and maximum values for the scalar colormap
ps_tutte.add_scalar_quantity("area distortion", E_area, defined_on='faces', cmap='reds', vminmax=(0,2), enabled=True)
ps_tutte.add_scalar_quantity("conformal distortion", E_conformal, defined_on='faces', cmap='reds', vminmax=(0,2), enabled=False)
ps_tutte.add_scalar_quantity("isometric distortion", E_isometric, defined_on='faces', cmap='reds', vminmax=(0,10), enabled=False)

ps.reset_camera_to_home_view()
ps.show()

Feel free to try experimenting with your own choice of weights! 

In [ ]:
# your_custom_weights = ...
# L, Lb = setup_parameterization_matrices(pyramid_vs, boundary_idxs, valid_edges, weights=your_custom_weights)
# your_custom_solution = np.linalg.solve(L, Lb @ boundary_positions)

# Don't forget to bring back the pinned coordinates into the solution
# your_custom_uv = np.zeros_like(pyramid_vs[:, :2])
# your_custom_uv[boundary_idxs] = boundary_positions
# your_custom_uv[pred_idxs] = your_custom_solution[pred_idxs]